In [2]:
import json 
import re

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN 


In [3]:
stopword = thai_stopwords()
def clean_tag(text):
  text = re.sub('<.*?>','',text)
  return text
def complete_clean(text :str):
  
  # Table for emoticon
  emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002500-\U00002BEF"  # chinese char
                            u"\U00002702-\U000027B0"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            u"\U0001f926-\U0001f937"
                            u"\U00010000-\U0010ffff"
                            u"\u2640-\u2642"
                            u"\u2600-\u2B55"
                            u"\u200d"
                            u"\u23cf"
                            u"\u23e9"
                            u"\u231a"
                            u"\ufe0f"  # dingbats
                            u"\u3030"
                            "]+", flags=re.UNICODE)

  # clean paunsuation
  nopunc_text = re.sub(r"[·“”\"\\,@\'?\$%_\[\]()/*+<=>!`~{|}^:;,-.&#]", "", text, flags=re.I)

  # clean emoticon
  clean_text = emoji_pattern.sub(r'', nopunc_text)

  # clean tag html
  cleantext_text = clean_tag(clean_text)

  return cleantext_text
def notdetermine(x):
  return x not in stopword
def removeStopword (arr):
  somelist = [x for x in arr if notdetermine(x)]
  return somelist
def tokenization(text):
  # token the sentence
  token_text = word_tokenize(text, engine="newmm", keep_whitespace=False)
  return token_text


In [4]:

with open(f'../data/DataTank.json') as f:
  alldata = json.load(f)


In [5]:
dataPreProcessings = {}
for kratoo in alldata:
  kratooContent = alldata[kratoo]

  # preprocessing
  titleDes = kratooContent['title']+kratooContent['desc']
    # clean process
  CleanTxt = complete_clean(titleDes)
  # data token 
  dataToken = tokenization(CleanTxt)
  # remove stop word
  bareText = removeStopword(dataToken)

  dataPreProcessing  = kratooContent
  dataPreProcessing['title'] = bareText
  dataPreProcessing['desc'] = ''

  dataPreProcessings[dataPreProcessing['tid']] = dataPreProcessing

# save datapreprocessing
with open(f"../data/dataPreProcess.json",mode='w') as datasave:
  content = json.dump(dataPreProcessings, datasave, ensure_ascii=False)

print('Data preprocess finish!!!!!!')

Data preprocess finish!!!!!!


In [6]:
############# TF-idf #############

listData = []
for kratoo in dataPreProcessings:
  listData.append(dataPreProcessings[kratoo]['title'])

tokens_listData = [','.join(tkn) for tkn in listData]
tfidf = TfidfVectorizer(analyzer=lambda x:x.split(','),)
try:
  %time tfidf_resutl = tfidf.fit_transform(tokens_listData).todense()
except:
  print('cant do tf-idf')

print('TFIDF finish!!!!!!')


CPU times: user 179 ms, sys: 83.1 ms, total: 263 ms
Wall time: 308 ms
TFIDF finish!!!!!!


In [1]:
############# DBscan #############

dbscan = DBSCAN(eps=50, min_samples=3)
db = dbscan.fit(tfidf_resutl) 
finalResult = (db.labels_).tolist()
print(len(finalResult))

NameError: name 'DBSCAN' is not defined